In [2]:
# Import the two main libraries
from bs4 import BeautifulSoup       # to process html
import requests
import urllib.request,sys,time
import pandas as pd
from tqdm import tqdm
import pickle
import datetime

In [3]:
import warnings
warnings.filterwarnings("ignore")

This notebook is a fork of the one [about the Sun](./scraper_sun.ipynb) for more information look at that one before.

In [4]:
#get all the links of the articles

saved_links = []
for page in tqdm(range(1,11)):

    url = 'https://sputnikglobe.com/search/?query=Russia-Ukraine+war='+str(page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all('a',{'_search-result'})
    
    for link in links:
        saved_links.append(link['href'].strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.06s/it]


In [5]:
file_name = "bbc_links.pkl"

open_file = open(file_name, "wb")
pickle.dump(saved_links, open_file)
open_file.close()

In [6]:
# to load
open_file = open("bbc_links.pkl", "rb")
saved_links = pickle.load(open_file)
open_file.close()

In [7]:
#creating the dataframe

BbcArticles = pd.DataFrame({'title': pd.Series([], dtype='string'),
                             'author': pd.Series([], dtype='string'),
                             'date': pd.Series([], dtype='float'),
                             'text': pd.Series([], dtype='string'),
                             })


The main difference with the one of the Sun is that not all the link that we have of the BBC are about articles, so sometimes beautiful soup fail while is searching f

In [13]:
print(saved_links)

['https://www.economist.com/ukraine-crisis', 'https://www.economist.com/graphic-detail/2022/03/21/in-russias-war-in-ukraine-both-sides-have-attacked-water-supplies', 'https://www.economist.com/briefing/2022/06/30/does-a-protracted-conflict-favour-russia-or-ukraine', 'https://www.economist.com/the-world-ahead/2022/11/14/three-scenarios-for-how-war-in-ukraine-could-play-out', 'https://www.economist.com/finance-and-economics/2023/03/30/how-rare-gas-supply-adapted-to-russias-war', 'https://www.economist.com/graphic-detail/2023/03/08/how-many-russians-have-been-killed-in-ukraine', 'https://www.economist.com/business/russias-war-is-creating-corporate-winners-and-losers/21808223', 'https://www.economist.com/international/2022/04/04/how-if-at-all-might-russia-be-punished-for-its-war-crimes-in-ukraine', 'https://www.economist.com/the-economist-explains/2022/02/22/what-is-hybrid-war-and-is-russia-waging-it-in-ukraine', 'https://www.economist.com/europe/moldova-is-trying-to-stay-out-of-russias-wa

In [15]:
for i,link in tqdm(enumerate(saved_links)):
    
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    try: # the try except is made to handle the problem that some links of the bbc aren't article, so when beautiful soup fail to load the article tag the news is skipped
        # get article tag
        article = soup.find_all("article")[0]

        #get title
        title = article.find_all("h1")[0].get_text()

        #get author
        author = ''

        #get date
        date = article.time['datetime']
        date = pd.to_datetime(date).strftime('%d/%m/%y')

        #get text
        text = ''
        for p in article.find_all('p'):
            text += p.text

        BbcArticles = BbcArticles.append({"title": title, "author": author, "date": date, "text": text}, ignore_index=True)
    except:
        pass

100it [01:40,  1.01s/it]


In [16]:
BbcArticles

,title,author,date,text
0,"In Russia’s war in Ukraine, both sides have at...",,21/03/22,Shortly after Vladimir Putin began his bloody ...
1,Does a protracted conflict favour Russia or Uk...,,30/06/22,"The typical war is short. Since 1815, the medi..."
2,Three scenarios for how war in Ukraine could p...,,14/11/22,"By Shashank Joshi: Defence editor, The Economi..."
3,How rare-gas supply adapted to Russia’s war,,30/03/23,Read more of our recent coverage of the Ukrain...
4,How many Russians have been killed in Ukraine?,,08/03/23,Read more of our recent coverage of the Ukrain...
...,...,...,...,...
94,The world’s deadliest war last year wasn’t in ...,,17/04/23,Fighter jets roar over Khartoum. Bombs rattle ...
95,Europe’s hard-hit east is pushing the toughest...,,07/05/22,"WHEN RUSSIA invaded Ukraine, many eastern Euro..."
96,How Putin’s war is causing global instability,,18/11/22,"By Robert Guest: Foreign editor, The Economist..."
97,War by any other name,,03/07/14,"A “NON-LINEAR” war, explains Natan Dubovitsky,..."


In [18]:
BbcArticles.to_csv("scraper_the_economist.csv")